In [5]:
# Pilar Fernandez Gayol
# Illinois Institute of Technology 
# ETS Ingenieros Informaticos, Universidad Politecnica de Madrid

<h1 style="font-size: 3em;">Integration and Tutorial of pmacctd Toolset on FABRIC Testbed</h1>

#  Introduction
The FABRIC testbed is a central hub for large-scale networking experimentation and research, offering a platform for exploring innovative networking technologies. pmacct, a set of open-source passive monitoring tools by Paolo Lucente, is widely used for network traffic monitoring. Integrating pmacct into FABRIC enhances its monitoring capabilities, enabling real-time traffic analysis in an advanced research setting. 

This project leverages multiple technologies to develop and execute experiments, integrating pmacct into FABRIC to advance network monitoring and experimentation. The goal is to provide real-world tools and settings for computer science and engineering students to explore and innovate in networking.

> Throughout this notebook we will show the integration of pmacctd in FABRIC, we will perform several experiments that show the multiple capabilities offered by this tool.

<div style="text-align: center;">
    <img src="./images/1.png" width="350px" alt="pmacctd integration on a FABRIC node"/>
</div>



# Setup and Installation

This section includes the necessary installations and configurations required to run the subsequent code. It ensures that all dependencies and environments are correctly set up.

## Creation of the slice

For this experiments we are going to create a slice named "pmacctd_slice" with two nodes (n1 and n2), each with specified resources, connected by a layer 2 network (L2Bridge). The architecture enables experimentation with network configurations and monitoring.

Each node uses an Ubuntu 20.04 image, which allows the installation of the pmacctd tool on it. It also provides a robust and flexible operating system for experimenting with different network configurations and monitoring setups.

<div style="text-align: center;">
    <img src="./images/2.png" width="400px"/>
</div>

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

# Initialize the FABRIC library manager
fablib = fablib_manager()

# Define the slice and node names
slice_name = "pmacctd_slice"
n1_name = "n1"
n2_name = "n2"

# Try to find an existing slice with the given name
slice = None
for slice_candidate in fablib.get_slices():
    if slice_candidate.get_name() == slice_name:
        slice = slice_candidate
        break

# If the slice does not exist, create a new one
if None == slice:
    image = 'default_ubuntu_20'
    site = 'MAX'
    slice = fablib.new_slice(name=slice_name)

    # Add the first node to the slice with specified resources
    n1 = slice.add_node(name=n1_name, image=image, cores=2, ram=4, disk=50, site=site)
    n1_iface1 = n1.add_component(model="NIC_Basic", name="iface1").get_interfaces()[0]

    # Add the second node to the slice with specified resources
    n2 = slice.add_node(name=n2_name, image=image, cores=2, ram=4, disk=50, site=site)
    n2_iface1 = n2.add_component(model="NIC_Basic", name="iface1").get_interfaces()[0]

    # Create a layer 2 network connecting both nodes
    net = slice.add_l2network(name="net", interfaces=[n1_iface1, n2_iface1], type="L2Bridge")
    slice.submit()

# Retrieve the nodes from the slice
n1 = slice.get_node(name=n1_name)
n2 = slice.get_node(name=n2_name)

# Display the slice information
slice.show()


## Configuration of the slice


##### Installing Essential Development and Network Tools on FABRIC Nodes
   
The following code updates the package lists and installs essential development tools (build-essential), network utilities (net-tools), and the Python Scapy library (python3-scapy) on the two nodes (n1 and n2).
These tools are necessary for compiling software, managing network configurations, and manipulating network packets, preparing the nodes for network experiments and monitoring tasks of pmacctd.



In [7]:


# For node n1
stdout, stderr = n1.execute(
    "sudo apt update -y -qq && sudo apt install -y build-essential && sudo apt install -y net-tools && sudo apt install -y python3-scapy",
    quiet=True
)

# For node n2
stdout, stderr = n2.execute(
    "sudo apt update -y -qq && sudo apt install -y build-essential && sudo apt install -y net-tools && sudo apt install -y python3-scapy",
    quiet=True
)
   

##### Testing SSH Connectivity and Retrieving Commands for FABRIC Nodes
   
This code tests the SSH connectivity on the two nodes (n1 and n2), printing whether SSH works on each node. 
It also retrieves and prints the SSH command for connecting to each node, these commands can be pasted on a terminal to gain access to the node ubuntu terminal for each node.

If the result is False, an error has occurred during the creation of the slice; please delete it and try to create it again.

In [ ]:
print("ssh works on n1: " + str(n1.test_ssh()))
print(n1.get_ssh_command())

print("ssh works on n2: " + str(n2.test_ssh()))
print(n2.get_ssh_command())

##### Displaying Network Interfaces on FABRIC Nodes
   
This code prints the network configuration of two nodes (n1 and n2) by executing the ifconfig command on each node.

In [ ]:
# Management vs Experiment interfaces.
print("ifconfig n1:")
n1.execute("ifconfig")
print("ifconfig n2:")
n2.execute("ifconfig")

##### Configuring IPv6 Addresses on the Nodes
This code configures IPv6 addresses for the interfaces on the two nodes (n1 and n2). It assigns local IPv6 addresses and then displays the interface configurations.

In [ ]:
from ipaddress import IPv6Address, IPv6Network

n1_iface1 = n1.get_interface(name = "n1-iface1-p1")
n2_iface1 = n2.get_interface(name = "n2-iface1-p1")

n1_iface1.ip_addr_add(addr = IPv6Address("fd3f:f209:c712::1"), subnet = IPv6Network("fd3f:f209:c712::/48"))
n2_iface1.ip_addr_add(addr = IPv6Address("fd3f:f209:c712::2"), subnet = IPv6Network("fd3f:f209:c712::/48"))
n1_iface1.show()
n2_iface1.show()
n1_ip = "fd3f:f209:c712::1"
n2_ip = "fd3f:f209:c712::2"

##### Configuring the Network Interface
This code configurates the network interfaces of the two nodes (n1 and n2), bringing up the network interfaces using the ifconfig command with root priviledges.

In [ ]:
n1.execute("sudo ifconfig " + n1_iface1.get_device_name() + " up")
n2.execute("sudo ifconfig " + n2_iface1.get_device_name() + " up")

##### IPv6 Ping Test Between Nodes
The following cell performs a single ICMPv6 ping test between network interfaces of nodes n1 and n2. 
The code executes these commands on both nodes to verify IPv6 connectivity between them.

In [ ]:
command = "ping -c 1 -I " + n1_iface1.get_device_name() + " fd3f:f209:c712::2"
print(command)
n1.execute(command) 

command2 = "ping -c 1 -I " + n2_iface1.get_device_name() + " fd3f:f209:c712::1"
print(command2)
n2.execute(command2)

##### Transfering files to the nodes
The following cell defines two functions that will be subsequently used on the notebook to upload and download files between the JypyterHub workspace and the nodes.

For performing each action it is necessary to place the file on the workspace, and specify the node and the file (its route may be added too).

You just need to run this cell once to use the defined functions.

In [16]:
# Transferring files between your JupyterHub and your node

# NOTE could use n1.upload_file(...)
def cmd_upload_file_to(n, filename):
    scp_pre_command = (n.get_ssh_command()).split(" ")
    scp_pre_command[0] = 'scp'
    scp_pre_command[5] = scp_pre_command[5].split("@")
    scp_pre_command[5] = scp_pre_command[5][0] + "@[" + scp_pre_command[5][1] + "]:~/"
    scp_pre_command.insert(5, filename)
    return ' '.join(scp_pre_command)

def cmd_download_file_from(n, filename):
    scp_pre_command = (n.get_ssh_command()).split(" ")
    scp_pre_command[0] = 'scp'
    scp_pre_command[5] = scp_pre_command[5].split("@")
    scp_pre_command[5] = scp_pre_command[5][0] + "@[" + scp_pre_command[5][1] + "]:~/" + filename
    scp_pre_command.append(".")
    return ' '.join(scp_pre_command)

## Installation

This section includes the necessary installations and configurations required to run the pmacctd tool and to perform actions and experiments on it. It also ensures that all dependencies and environments are correctly set up.

First, the zip files of the tool and the jansson library must be added to the workspace so that they can be transferred to the nodes.

> Note: You can find the zip files on:
>> pmacctd: http://www.pmacct.net/#downloads
>> jansson library: https://digip.org/jansson/releases/


> Note: You may need to change the names of the files in the cell 

In [ ]:
#Upload the zip files to the nodes, you may paste the commands on a terminal to upload the files to the nodes.
print(cmd_upload_file_to(n1, "pmacct-1.7.8.zip"))
print(cmd_upload_file_to(n1, "jansson-2.13.tar.gz"))
print(cmd_upload_file_to(n2, "pmacct-1.7.8.zip"))
print(cmd_upload_file_to(n2, "jansson-2.13.tar.gz"))

> The following code unzips the files loaded to the nodes, but first installs the neccessary software packages.


In [21]:
# Unzip both files
# First, install zip in the nodes
stdout, stderr = n1.execute(
    "sudo apt-get install -y zip && unzip pmacct-1.7.8.zip && tar -xzf jansson-2.13.tar.gz",
    quiet=True
)

stdout, stderr = n2.execute(
    "sudo apt-get install -y zip && unzip pmacct-1.7.8.zip && tar -xzf jansson-2.13.tar.gz",
    quiet=True
)


##### Jansson library installation
The following code installs the jansson library on the nodes. This library is needed so that the pmacctd tool can diplay the information of the results on the experiments in a JSON format.


In [22]:
## Installation of jansson library
stdout, stderr = n1.execute(
    "cd jansson-2.13 && ./configure && make && sudo make install",
    quiet=True
)
stdout, stderr = n2.execute(
    "cd jansson-2.13 && ./configure && make && sudo make install",
    quiet=True
)


##### pmacctd installation 
The following code installs the neccesary dependencies for the use of the tool and then installs it following the commands.

1. Place on the pmacct folder (cd pmacct-1.7.8)
2. Run the command ./autogen.sh
3. Run the command ./configure --enable-jansson (so that the results can be created on a JSON format too)
4. Run the command make
5. Run the command make install (superuser privileges)


In [23]:
## Installation of pmacctd 
stdout, stderr = n1.execute(
    "sudo apt-get install -y libpcap-dev pkg-config libtool autoconf automake make bash libstdc++6 g++ libjansson4",
    quiet=True
)
stdout, stderr = n1.execute(
    "cd pmacct-1.7.8 && ./autogen.sh && ./configure --enable-jansson && make && sudo make install",
    quiet=True
)


stdout, stderr = n2.execute(
    "sudo apt-get install -y libpcap-dev pkg-config libtool autoconf automake make bash libstdc++6 g++ libjansson4",
    quiet=True
)
stdout, stderr = n2.execute(
    "cd pmacct-1.7.8 && ./autogen.sh && ./configure --enable-jansson && make && sudo make install",
    quiet=True
)


# Experiments and use of the tool
The following sections describe different experiments performed on the pmacctd tool to know more about its capabilities. The experiments are performed both using .pcap files and live traffic captured by pmacctd.

> You can use any .pcap file but I would be using the following one for the experiments gmail.pcapng.cap, which can be found in: https://packetlife.net/captures/
You need to first upload it to the node using the upload function and then move it to the pmacctd folder: print(cmd_upload_file_to(n1, "gmail.pcapng.cap"))

> We are going to use different configurations of the tool so that it works properly. These configurations can be found on the configuration-files folder and placed on the Jupyter Workspace. You can play with them so that you can learn how the tool works.

## Experiment of the pmacctd tool using a pcap file.
In this section we are going to perform three different experiments using the same pcap file but with different configurations of the tool. We are going to show the results obtained from processing the pmacctd tool on a pcap file in three different formats: CSV, JSON and .txt. 

For the execution of the tool a configuration file is needed, in this section the following fields are necessary: 

daemonize: false \
debug: true \
pcap_savefile: file.pcap \
aggregate: src_host, dst_host, src_port, dst_port, proto, \
plugin_buffer_size: 4096 \
plugins: print \
print_output: csv/formatted/json \
print_output_file: result_file.format \
print_history_roundoff: m   


 - daemonize: This field indicates whether the process should run in the background or not. In this case, will set to false, meaning the process will run in the foreground.
  - debug: This field controls the debug mode. When set to true, it enables debugging output, which is be helpful for these first exeperiments.
  - pcap_savefile: This field specifies the path where the tool obtains the data. It will set to $PATH_of_pmacct_tool/pmacct-1.7.8/file.pcap. The file contains captured network traffic data.
  - aggregate: This field specifies the fields by which to aggregate the captured data. In this case, it will aggregate data based on source host, destination host, source port, destination port, and protocol.
    > The output would look like this:
    <div style="text-align: center;">
    <img src="./images/3.png" width="1000px"/>
</div>
  - plugin_buffer_size: This field defines the buffer size used by plugins. 
  - plugins: This field specifies the plugins to be used. In this example, the print plugin is specified, which likely indicates that the captured data will be printed. 
  - print_output: This field specifies the output format for the printed data. It will be set to csv for the first experiment since we want our output in this format.
  - print_history_roundoff: This field specifies the rounding-off interval for historical data. 

### CSV format
The configuration file will have the following data: 

daemonize: false \
debug: true \
pcap_savefile: gmail.pcapng.cap \
aggregate: src_host, dst_host, src_port, dst_port, proto, \
plugin_buffer_size: 4096 \
plugins: print \
print_output: csv \
print_output_file: result_pcap_csv.csv \
print_history_roundoff: m 

For this, we are going to upload to the node the following configuration file pcap_csv.conf which contains the information shown.
The gmail.pcapng.cap is already uploaded on the node and placed on the pmacctd folder.

In [ ]:
# You need to paste the resulting commands into the terminal to upload the files
print(cmd_upload_file_to(n1, "configuration-files/pcap_csv.conf"))
print(cmd_upload_file_to(n2, "configuration-files/pcap_csv.conf"))

##### Execution

1. Move the configuration file to the pmacctd folder.
2. Create the result csv file and move it to the pmacctd folder.
3. Execute pmacctd.
4. Show the results.
5. You can execute the last cell of the section to download the csv result file into your computer.

In [ ]:
#Move configuration file to pmacctd directory, execute experiment and see results
n1.execute("mv pcap_csv.conf pmacct-1.7.8")
n2.execute("mv pcap_csv.conf pmacct-1.7.8")
n1.execute("touch result_pcap_csv.csv && mv result_pcap_csv.csv pmacct-1.7.8")
n1.execute("cd pmacct-1.7.8 && sudo pmacctd -f pcap_csv.conf && cat result_pcap_csv.csv")

In [ ]:
print(cmd_download_file_from(n1, "pmacct-1.7.8/result_pcap_csv.csv"))

### JSON format
The configuration file will have the following data: 

daemonize: false \
debug: true \
pcap_savefile: gmail.pcapng.cap \
aggregate: src_host, dst_host, src_port, dst_port, proto, \
plugin_buffer_size: 4096 \
plugins: print \
print_output: csv \
print_output_file: result_pcap_json.json \
print_history_roundoff: m 

For this, we are going to upload to the node the following configuration file pcap_json.conf which contains the information shown.
The gmail.pcapng.cap is already uploaded on the node and placed on the pmacctd folder.

In [ ]:
# You need to paste the resulting commands into the terminal to upload the files
print(cmd_upload_file_to(n1, "configuration-files/pcap_json.conf"))
print(cmd_upload_file_to(n2, "configuration-files/pcap_json.conf"))

##### Execution

1. Move the configuration file to the pmacctd folder.
2. Create the result .json file and move it to the pmacctd folder.
3. Execute pmacctd.
4. Show the results.
5. You can execute the last cell of the section to download the json result file into your computer.

In [ ]:
#Move configuration file to pmacctd directory, execute experiment and see results
n1.execute("mv pcap_json.conf pmacct-1.7.8")
n2.execute("mv pcap_json.conf pmacct-1.7.8")
n1.execute("touch result_pcap_json.json && mv result_pcap_json.json pmacct-1.7.8")
n1.execute("cd pmacct-1.7.8 && sudo pmacctd -f pcap_json.conf && cat result_pcap_json.json")

In [ ]:
print(cmd_download_file_from(n1, "pmacct-1.7.8/result_pcap_json.json"))

### TXT Format



The configuration file will have the following data: \

daemonize: false \
debug: true \
pcap_savefile: gmail.pcapng.cap \
aggregate: src_host, dst_host, src_port, dst_port, proto, \
plugin_buffer_size: 4096 \
plugins: print \
print_output: formatted \
print_output_file: result_pcap_formatted.txt\
print_history_roundoff: m 

For this, we are going to upload to the node the following configuration file pcap_formatted.conf which contains the information shown.
The gmail.pcapng.cap is already uploaded on the node and placed on the pmacctd folder.


In [ ]:
# You need to paste the resulting commands into the terminal to upload the files
print(cmd_upload_file_to(n1, "configuration-files/pcap_formatted.conf"))
print(cmd_upload_file_to(n2, "configuration-files/pcap_formatted.conf"))

##### Execution

1. Move the configuration file to the pmacctd folder.
2. Create the result txt file and move it to the pmacctd folder.
3. Execute pmacctd.
4. Show the results.
5. You can execute the last cell of the section to download the .txt result file into your computer.

In [ ]:
#Move configuration file to pmacctd directory, execute experiment and see results
n1.execute("mv pcap_formatted.conf pmacct-1.7.8")
n2.execute("mv pcap_formatted.conf pmacct-1.7.8")
n1.execute("touch result_pcap_formatted.txt && mv result_pcap_formatted.txt pmacct-1.7.8")
n1.execute("cd pmacct-1.7.8 && sudo pmacctd -f pcap_formatted.conf && cat result_pcap_formatted.txt")

In [ ]:
print(cmd_download_file_from(n1, "pmacct-1.7.8/result_pcap_formatted.txt"))

### Customization 

Pmacctd offers an important characteristic, which is that enables the user to choose which fields wants to obtain from the captured data. This can be done using the "aggregate" field on the configuration file.

All the available aggregate fields are: src_mac, dst_mac, vlan, in_vlan, out_vlan, cos, etype, src_host, dst_host,src_net, dst_net, src_mask, dst_mask, src_as,    dst_as, src_port, dst_port, tos, proto, none, sum_mac, sum_host, sum_net, sum_as, sum_port, flows, tag, tag2, label, class, tcpflags, in_iface, out_iface, std_comm, ext_comm, lrg_comm, as_path, peer_src_ip, peer_dst_ip, peer_src_as, peer_dst_as, local_pref, med, dst_roa, src_std_comm, src_ext_comm, src_lrg_comm, src_as_path, src_local_pref, src_med, src_roa, mpls_vpn_rd, mpls_pw_id, mpls_label_top, mpls_label_bottom, mpls_label_stack, sampling_rate, sampling_direction, src_host_country, dst_host_country, src_host_pocode, dst_host_pocode, src_host_coords, dst_host_coords, nat_event, fw_event, post_nat_src_host, post_nat_dst_host, post_nat_src_port, post_nat_dst_port, tunnel_src_mac, tunnel_dst_mac, tunnel_src_host, tunnel_dst_host, tunnel_proto, tunnel_tos, tunnel_src_port, tunnel_dst_port, tunnel_tcpflags, fwd_status, vxlan, timestamp_start, timestamp_end, timestamp_arrival, timestamp_export, export_proto_seqno, export_proto_version, export_proto_sysid.

Some of them may not be supported by some packets but we can make a small example to test some of them (the most used ones).
 
daemonize: false \
debug: true \
pcap_interface: any \
aggregate: src_host, dst_host, src_port, dst_port, proto, src_mac, dst_mac, etype, in_iface, out_iface, \
plugin_buffer_size: 4096 \
plugins: print \
print_output: formatted \
print_output_file: result_custom.txt \
print_history_roundoff: m 


For this, we are going to upload to the node the following configuration file live_custom.conf which contains the information shown.
The gmail.pcapng.cap is already uploaded on the node and placed on the pmacctd folder.

In [ ]:
# You need to paste the resulting commands into the terminal to upload the files
print(cmd_upload_file_to(n1, "configuration-files/pcap_custom.conf"))
print(cmd_upload_file_to(n2, "configuration-files/pcap_custom.conf"))

##### Execution

1. Move the configuration file to the pmacctd folder.
2. Create the result txt file and move it to the pmacctd folder.
3. Execute pmacctd.
4. Show the results.
5. You can execute the last cell of the section to download the .txt result file into your computer.

In [ ]:
#Move configuration file to pmacctd directory, execute experiment and see results
n1.execute("touch result_custom.txt")
n1.execute("mv pcap_custom.conf result_custom.txt pmacct-1.7.8")
n2.execute("mv pcap_custom.conf pmacct-1.7.8")
n1.execute("cd pmacct-1.7.8 && sudo pmacctd -f pcap_custom.conf && cat result_custom.txt")

In [ ]:
#Download results
print(cmd_download_file_from(n1, "pmacct-1.7.8/result_custom.txt"))

### Filtering data
Pmacctd allow filtering the data captured using the pcap_filter field on the configuration file.

This filter must be written as in tcpdump, in order to work properly. The following configuration file shows an example of it.
 
daemonize: false \
debug: true \
pcap_savefile: gmail.pcapng.cap \
pcap_filter: tcp \
aggregate: src_host, dst_host, src_port, dst_port, proto, \
plugin_buffer_size: 4096 \
plugins: print \
print_output: csv \
print_output_file: result_pcap_filter.csv \
print_history_roundoff: m 


For this, we are going to upload to the node the following configuration file pcap_filter.conf which contains the information shown.
The gmail.pcapng.cap is already uploaded on the node and placed on the pmacctd folder.

The expected output is a csv file which only contains information about tcp packets.

In [ ]:
# You need to paste the resulting commands into the terminal to upload the files
print(cmd_upload_file_to(n1, "configuration-files/pcap_filter.conf"))
print(cmd_upload_file_to(n2, "configuration-files/pcap_filter.conf"))

##### Execution

1. Move the configuration file to the pmacctd folder.
2. Create the result csv file and move it to the pmacctd folder.
3. Execute pmacctd.
4. Show the results.
5. You can execute the last cell of the section to download the .csv result file into your computer.

In [ ]:
#Move configuration file to pmacctd directory, execute experiment and see results
n1.execute("touch result_pcap_filter.csv")
n1.execute("mv pcap_filter.conf result_pcap_filter.csv pmacct-1.7.8")
n1.execute("cd pmacct-1.7.8 && sudo pmacctd -f pcap_filter.conf && cat result_pcap_filter.csv")

In [ ]:
#Download results
print(cmd_download_file_from(n1, "pmacct-1.7.8/result_pcap_filter.csv"))

## Experiment of the pmacctd tool using live traffic
In this section we are going to perform an experiment using live traffic sending ICMP packets between nodes by using the "ping" command. We are going to show the results obtained from processing the pmacctd tool on a file in the formatted different format. This format can be changed to any different one as we did on the last section, you just have to change its value on the configuration file and re-uploaded it to the node. 

For the execution of the tool a configuration file is needed, in this section the forllowing fields are necessary: 

daemonize: false \
debug: true \
pcap_interface: any/eth3s0 \
aggregate: src_host, dst_host, src_port, dst_port, proto, \
plugin_buffer_size: 4096 \
plugins: print \
print_output: csv/formatted/json \
print_output_file: result_file.format \
print_history_roundoff: m   

  - pcap_interface: This field specifies the interface where the tool captures the data from. The value can be "any" or the name of the interface itself.
    
For this, we are going to upload to the node the following configuration file live.conf which contains the information shown.

In [ ]:
# You need to paste the resulting commands into the terminal to upload the files
print(cmd_upload_file_to(n1, "configuration-files/live.conf"))
print(cmd_upload_file_to(n2, "configuration-files/live.conf"))

##### Execution

1. Create the result result file.
2. Move the configuration file and the result file to the pmacctd folder.
3. Create live traffic betweeen the nodes.
4. Execute pmacctd.
5. Show the results.
6. You can execute the last cell of the section to download the .txt result file into your computer.

In [ ]:
#Move configuration file to pmacctd directory, execute experiment and see results
n1.execute("touch result_live.txt")
n1.execute("mv live.conf result_live.txt pmacct-1.7.8")
n2.execute("mv live.conf pmacct-1.7.8")

##### Concurrent Execution
The following cell uses the Python's threading module for concurrent execution of tasks. Two threads are created to run functions run_pmacctd on nodes n1 and n2, executing commands command_n1 and command_n2 respectively. This allows simultaneous operation of network monitoring (pmacctd) and IPv6 ping testing. The use of join() ensures synchronization, waiting for completion of the second thread before ending execution.

This is used because when using live traffic pmacctd acts as a demon so it needs to be stopped manually. Since we are testing the live traffic capabilities of pmacctd, we can use threads so that first the node n1 starts the daemon of pmacctd and then n2 pings 20 times to n1. Then when the second command finishes also the pmacctd tool stops so that we can see the results.

In [ ]:
import threading 
def run_pmacctd(node, command):
    result = node.execute(command)
    print(result)
command_n1 = "cd pmacct-1.7.8 && sudo pmacctd -f live.conf"
command_n2 = "ping -c 20 " + n1_ip

thread_h1 = threading.Thread(target=run_pmacctd, args=(n1, command_n1))
thread_h2 = threading.Thread(target=run_pmacctd, args=(n2, command_n2))

thread_h1.start()
thread_h2.start()

thread_h2.join()

In [ ]:
#Show results
n1.execute("cd pmacct-1.7.8 && cat result_live.txt")
print(cmd_download_file_from(n1, "pmacct-1.7.8/result_live.txt"))

### Filtering data
The pmacctd tool allows for data filtering through the pcap_filter field in its configuration file. This feature is particularly useful when working with live traffic, as it enables you to focus on specific data of interest or identify anomalies. By applying filters, you can streamline your data analysis, making it easier to pinpoint relevant information or troubleshoot issues.

This filter must be written as in tcpdump, in order to work properly. The following configuration file shows an example of it. This time we are going to implement a mmore complex filter. We are going to filter by the ip of the host and the type protocol. We can also delete that information from the aggregation field since all the data collected will present the same data of them. 
 
daemonize: false \
debug: true \
pcap_interface: any \
pcap_filter: icmp6 and src host fd3f:f209:c712::2 \
aggregate: src_port, dst_port, etype, in_iface, out_iface, \
plugin_buffer_size: 4096 \
plugins: print \
print_output: csv \
print_output_file: result_live_filter.csv \
print_history_roundoff: m 


For this, we are going to upload to the node the following configuration file live_filter.conf which contains the information shown.
The gmail.pcapng.cap is already uploaded on the node and placed on the pmacctd folder.

The expected output is a csv file which only contains information about icmp packets with source IP address fd3f:f209:c712::2 (n2).

In [ ]:
# You need to paste the resulting commands into the terminal to upload the files
print(cmd_upload_file_to(n1, "configuration-files/live_filter.conf"))
print(cmd_upload_file_to(n2, "configuration-files/live_filter.conf"))

##### Execution

1. Create the result result file.
2. Move the configuration file and the result file to the pmacctd folder.
3. Create live traffic betweeen the nodes.
4. Execute pmacctd.
5. Show the results.
6. You can execute the last cell of the section to download the .csv result file into your computer.

In [ ]:
#Move configuration file to pmacctd directory, execute experiment and see results
n1.execute("touch result_live_filter.csv")
n1.execute("mv live_filter.conf result_live_filter.csv pmacct-1.7.8")

##### Concurrent Execution
The following cell uses the Python's threading module for concurrent execution of tasks. Two threads are created to run functions run_pmacctd on nodes n1 and n2, executing commands command_n1 and command_n2 respectively. This allows simultaneous operation of network monitoring (pmacctd) and IPv6 ping testing. The use of join() ensures synchronization, waiting for completion of the second thread before ending execution.

This is used because when using live traffic pmacctd acts as a demon so it needs to be stopped manually. Since we are testing the live traffic capabilities of pmacctd, we can use threads so that first the node n1 starts the daemon of pmacctd and then n2 pings 20 times to n1. Then when the second command finishes also the pmacctd tool stops so that we can see the results.

In [ ]:
import threading 
def run_pmacctd(node, command):
    result = node.execute(command)
    print(result)
command_n1 = "cd pmacct-1.7.8 && sudo pmacctd -f live_filter.conf"
command_n2 = "ping -c 20 " + n1_ip

thread_h1 = threading.Thread(target=run_pmacctd, args=(n1, command_n1))
thread_h2 = threading.Thread(target=run_pmacctd, args=(n2, command_n2))

thread_h1.start()
thread_h2.start()

thread_h2.join()

In [ ]:
# Show results
n1.execute("cd pmacct-1.7.8 && cat result_live_filter.csv")

In [ ]:
# Download results
print(cmd_download_file_from(n1, "pmacct-1.7.8/result_live_filter.csv"))

## Experiment of the pmacctd tool using live traffic with iPerf

In this section we are going to continue to use live traffic to test the pmacctd tool, but this time we are going to use iPerf. 
iPerf is a network performance testing tool used to measure bandwidth between two hosts over a TCP or UDP connection. iPerf works by establishing a client-server model where one instance of iPerf acts as the server and another as the client. 

The objective of this is to test the pmacctd tool this time with UDP and TCP packets, instead of ICMP ones. 

For the execution of the tool a configuration file is needed, in this section the following fields are necessary: 

daemonize: false \
debug: true \
pcap_interface: any \
aggregate: src_host, dst_host, src_port, dst_port, proto, \
plugin_buffer_size: 4096 \
plugins: print \
print_output: csv/formatted/json \
print_output_file: result_file.format \
print_history_roundoff: m 


For this, we are going to upload to the node the following configuration file iperf.conf which contains the information shown.

In [ ]:
# You need to paste the resulting commands into the terminal to upload the files
print(cmd_upload_file_to(n1, "configuration-files/iperf.conf"))
print(cmd_upload_file_to(n2, "configuration-files/iperf.conf"))

> We need to install iperf on the nodes, since we use IPV6 we will use iperf3, which supports it.

In [59]:
#Install iperf3
stdout, stderr = n1.execute(
    "sudo apt-get install -y iperf3",
    quiet=True
)
stdout, stderr = n2.execute(
    "sudo apt-get install -y iperf3",
    quiet=True
)

##### Execution

1. Create the result result file.
2. Move the configuration file and the result file to the pmacctd folder.
3. Create live traffic betweeen the nodes.
4. Execute pmacctd.
5. Show the results.
6. You can execute the last cell of the section to download the .txt result file into your computer.

In [ ]:
#Move configuration file to pmacctd directory, execute experiment and see results
n1.execute("touch result_iperf.csv")
n1.execute("mv iperf.conf result_iperf.csv pmacct-1.7.8")
n2.execute("mv iperf.conf pmacct-1.7.8")

##### iPerf

This time we are going to use the same thread approach to test the tool, but using iPerf. n1 will act as the server node and n2 wil act as the client, but first we need to execute the daemon tool on the server node, which will finish when the last command finishes.

> Note: It must work, but if not, you will need to look for the PID of the process by getting into the terminal of the node and put the command ps aux | grep iperf and then use the command sudo kill -9 "PID" so that the address gets free and you can execute the iPerf again.

In [ ]:
import threading

def run_iperf(node, command):
    result = node.execute(command)
    print(result)
command = "cd pmacct-1.7.8 && sudo pmacctd -f iperf.conf"
command_n1 = "iperf3 -s"
command_n2 = "iperf3 -c " + n1_ip +  " -6"

thread = threading.Thread(target=run_iperf, args=(n1, command))
thread_n1 = threading.Thread(target=run_iperf, args=(n1, command_n1))
thread_n2 = threading.Thread(target=run_iperf, args=(n2, command_n2))

thread.start()
thread_n1.start()
thread_n2.start()

thread_n2.join()

In [ ]:
#Show results
n1.execute("cd pmacct-1.7.8 && cat result_iperf.csv")

In [ ]:
print(cmd_download_file_from(n1, "pmacct-1.7.8/result_iperf.csv"))

## Experiment of the pmacctd tool using a pcap file with IPFIX flows

IPFIX (IP Flow Information Export) flows are a standard method for exporting detailed information about IP traffic flows from network devices like routers and switches. They capture metadata including packet counts, timestamps, protocol types, ports, source and destination IP addresses, and so on. Security, traffic analysis, and network monitoring are all done with this data. IPFIX assists in anomaly detection and performance optimization. It is also a more advanced and flexible version of NetFlow, allowing for more detailed and customizable flow information export.

We are going to first work producing IPFIX flows with pmacctd using a pcap file, this will include first print the result on a terminal, then on a log file at pmacctd then we are going to create an .ipfix file which will be opened and analyzed with tshark and lastly we are going to send the flows to a collector using the Netflow Python library.

### Print the result on the terminal

For this first and easiest experiment we will use the following configuration file: 

daemonize: false \
debug: true \
pcap_savefile: gmail.pcapng.cap \
plugins: nfprobe, print \
nfprobe_version: 10 

- nfprobe: This plugin is used for exporting network flow data using the NetFlow/IPFIX protocol.
- print: This plugin is used for printing the collected data, in this case to standard output since we have not specified an output file.
- nfprobe_version: 10 refers to IPFIX, which is also known as NetFlow version 10. 

In [ ]:
# You need to paste the resulting commands into the terminal to upload the files
print(cmd_upload_file_to(n1, "configuration-files/ipfix_1.conf"))
print(cmd_upload_file_to(n2, "configuration-files/ipfix_1.conf"))

##### Execution

1. Move the configuration file to the pmacctd folder.
2. Execute pmacctd.
3. Show the results.

In [ ]:
#Move configuration file to pmacctd directory, execute experiment and see results
n1.execute("mv ipfix_1.conf pmacct-1.7.8")
n2.execute("mv ipfix_1.conf pmacct-1.7.8")

In [ ]:
n1.execute("cd pmacct-1.7.8 && sudo pmacctd -f ipfix_1.conf")

### Print the result on a log file

For this second experiment we will use the following configuration file: 

daemonize: false \
debug: true \
pcap_savefile: gmail.pcapng.cap \
plugins: nfprobe, print \
nfprobe_version: 10 \
print_output_file: ipfix_2.log

- print: This plugin is used for printing the collected data, in this case to a log file since we have specified an output file.
- print_output_file: refers to the log output file.

In [ ]:
# You need to paste the resulting commands into the terminal to upload the files
print(cmd_upload_file_to(n1, "configuration-files/ipfix_2.conf"))
print(cmd_upload_file_to(n2, "configuration-files/ipfix_2.conf"))

##### Execution

1. Create the result result log file.
2. Move the configuration file and the result file to the pmacctd folder.
3. Execute pmacctd.
4. Show the results.
5. You can execute the last cell of the section to download the .log result file into your computer.

In [ ]:
#Move configuration file to pmacctd directory, execute experiment and see results
n1.execute("touch ipfix_2.log")
n1.execute("mv ipfix_2.conf ipfix_2.log pmacct-1.7.8")
n2.execute("mv ipfix_2.conf pmacct-1.7.8")

In [ ]:
n1.execute("cd pmacct-1.7.8 && sudo pmacctd -f ipfix_2.conf")
n1.execute("cd pmacct-1.7.8 && cat ipfix_2.log")

In [ ]:
print(cmd_download_file_from(n1, "pmacct-1.7.8/ipfix_2.log"))

### Send the IPFIX flows to an .ipfix file 

For this experiment we will use the following configuration file: 

daemonize: false \
debug: true \
pcap_savefile: gmail.pcapng.cap \
plugins: nfprobe \
nfprobe_receiver: 127.0.0.1:4739 \
nfprobe_version: 10 

- nfprobe_receiver: it configures pmacct to send the collected network flow data to a local collector on IP address 127.0.0.1 (local) running on port 4739.

In [ ]:
# You need to paste the resulting commands into the terminal to upload the files
print(cmd_upload_file_to(n1, "configuration-files/ipfix_3.conf"))
print(cmd_upload_file_to(n2, "configuration-files/ipfix_3.conf"))

##### Execution

1. Create the result result IPFIX file.
2. Move the configuration file and the result file to the pmacctd folder.
3. Redirect the network flow of 127.0.0.1 port 4739 to the file
4. Execute pmacctd.
5. Show the results.
6. You can execute the last cell of the section to download the .log result file into your computer.

In [ ]:
#Move configuration file to pmacctd directory, execute experiment and see results
n1.execute("touch ipfix_flows_3.ipfix")
n1.execute("mv ipfix_3.conf ipfix_flows_3.ipfix pmacct-1.7.8")
n2.execute("mv ipfix_3.conf pmacct-1.7.8")

##### Redirecting the flow of 127.0.0.1 port 4739 to the file. 
We are going to do that using netcat. So we first need to download it and then use the following command to redirect the ipfix flows created to it: nc -u -l 4739 > pmacct-1.7.8/ipfix_flows_3.ipfix.

This command means that we are using the NetCat utility using UDP which listens for incoming connections on port 4739 and which redirects the output to the file specified on the path, in this cas ipfix_flows_3.ipfix


In [76]:
stdout, stderr = n1.execute(
    "sudo apt-get install netcat",
    quiet=True
)
stdout, stderr = n2.execute(
    "sudo apt-get install netcat",
    quiet=True
)

In [ ]:
import threading

def run_iperf(node, command):
    result = node.execute(command)
    print(result)
command_1 = "nc -u -l 4739 > pmacct-1.7.8/ipfix_flows_3.ipfix"
command_2 = "cd pmacct-1.7.8 && sudo pmacctd -f ipfix_3.conf"

thread_1 = threading.Thread(target=run_iperf, args=(n1, command_1))
thread_2 = threading.Thread(target=run_iperf, args=(n1, command_2))

thread_1.start()
thread_2.start()

thread_2.join()

In [ ]:
print(cmd_download_file_from(n1, "pmacct-1.7.8/ipfix_flows_3.ipfix"))

##### We need to download t-shark so that we can se the content of the flows in the result file. 
First, we need to download the file to inspect its content and the different flows. Since downloading TShark requires granting some permissions, we cannot automate this process. Please refer to the cell where the SSH command for node n1 is shown in the section. Execute that command in a terminal (so that that terminal is now the terminal of the node), and then run the following command: "sudo apt-get install -y tshark"


In [ ]:
#n1.execute("sudo apt-get install -y tshark")
#n2.execute("sudo apt-get install -y tshark")

##### Summary
The following command will read and display the packet summary information from the IPFIX file, showing a list of packets captured in the file.


In [ ]:
n1.execute("tshark -r pmacct-1.7.8/ipfix_flows_3.ipfix")

##### In detail 
This command reads the IPFIX file and provides a detailed hex and ASCII representation of the first packet's data. This is useful for in-depth analysis of the packet's structure and content.

You can change the number of the packet and execute the command again to inspect different ones.


In [ ]:
# Look one packet of a flow in detail
n1.execute("tshark -r pmacct-1.7.8/ipfix_flows_3.ipfix -x -c 1")

### Send the IPFIX flows to a collector

First, we need to download the netflow Python library from  https://pypi.org/project/netflow/ and place on the work directory. 
Also it is necessary to download python3 on the node: sudo apt-get install python3 to be able to use the library. Then we will uploade it to the nodes. 

In [ ]:
# Please, place the resulting command on a terminal
print(cmd_upload_file_to(n1, "netflow-0.12.2.tar.gz"))
print(cmd_upload_file_to(n2, "netflow-0.12.2.tar.gz"))      

In [ ]:
# Unzip the netflow library file, you can install the zip library by executing the command sudo apt-get zip, you can uncomment the following line and execute it
# If you follow this notebook from the beggining you should have the software package already installed.
#n1.execute("sudo apt-get zip")
n1.execute("tar -xzf netflow-0.12.2.tar.gz")
n2.execute("tar -xzf netflow-0.12.2.tar.gz")

It is necessary to download python3 and pip on the node to be able to use the library.

In [80]:
# Download pyython at the nodes and also netflow
stdout, stderr = n1.execute(
    "sudo apt-get install -y python3",
    quiet=True
)
stdout, stderr = n1.execute(
    "sudo apt-get install -y python3",
    quiet=True
)
stdout, stderr = n2.execute(
    "sudo apt-get install -y pip",
    quiet=True
)
stdout, stderr = n2.execute(
    "sudo apt-get install -y pip",
    quiet=True
)

We also need to install the netflow Python package so that the library works properly.

In [81]:
stdout, stderr = n1.execute(
    "pip install netflow",
    quiet=True
)
stdout, stderr = n2.execute(
    "pip install netflow",
    quiet=True
)

##### Configuration file

For this experiment we will use the following configuration file: 

daemonize: false \
debug: true \
pcap_savefile: gmail.pcapng.cap \
plugins: nfprobe \
nfprobe_receiver: 127.0.0.2:4739 \
nfprobe_version: 9 

- nfprobe_version: 9 refers to NetFlow, since we are using its Python version we need to specify it.

In [ ]:
# Please, place the resulting command on a terminal
print(cmd_upload_file_to(n1, "configuration-files/ipfix_4.conf"))
print(cmd_upload_file_to(n2, "configuration-files/ipfix_4.conf"))

##### Execution

1. Move the configuration file to the pmacctd folder.
2. Execute the python program collector.py specifying the host and the port 
3. Redirect the network flow of 127.0.0.2 port 4739 to the file
4. Execute pmacctd.
5. Show the results.
6. You can execute the last cell of the section to download the .log result file into your computer.

In [ ]:
n1.execute("mv ipfix_4.conf pmacct-1.7.8")
n2.execute("mv ipfix_4.conf pmacct-1.7.8")

In [ ]:
import threading

def run_iperf(node, command):
    result = node.execute(command)
    print(result)
command_1 = "cd netflow-0.12.2/netflow && python3 collector.py --host 127.0.0.2 --port 4739"
command_2 = "cd pmacct-1.7.8 && sudo pmacctd -f ipfix_4.conf"

thread_1 = threading.Thread(target=run_iperf, args=(n1, command_1))
thread_2 = threading.Thread(target=run_iperf, args=(n1, command_2))

thread_1.start()
thread_2.start()

thread_2.join()

##### Results

The results will be obtained on a .tar file, containing a JSON file placed on the netflow folder as you can see on the following picture:

<div style="text-align: center;">
    <img src="./images/4.png" width="700px"/>
</div>

You can unzip them using the command: gzip -d filename.gz

You can see the results using the command: cat filename

You can download the results using the command "print(cmd_download_file_from(n1, "filename"))" on a new cell of this Notebook 


##  Experiment of the pmacctd tool using live traffic with IPFIX flows
Now we are going to work producing IPFIX flows with pmacctd using the live traffic captured by pmacctd, this will include first printing the result on a terminal, then in a log file at pmacctd then we are going to create an .ipfix file which will be opened and analyzed with tshark and lastly we are going to send the flows to a collector using the Netflow Python library.

### Print the result on the terminal
For this first and easiest experiment we will use the following configuration file: 

daemonize: false \
debug: true \
pcap_interface: any \
plugins: nfprobe, print \
nfprobe_version: 10 \
nfprobe_receiver: localhost:4739

- nfprobe: This plugin is used for exporting network flow data using the NetFlow/IPFIX protocol.
- print: This plugin is used for printing the collected data, in this case to standard output since we have not specified an output file.
- nfprobe_version: 10 refers to IPFIX, which is also known as NetFlow version 10.
- pcap_interface: refers to the interface or interfaces where the tool will collect the data from.
- nfprobe_receiver: specifies the destination to which NetFlow/IPFIX data should be sent.

In [ ]:
# You need to paste the resulting commands into the terminal to upload the files
print(cmd_upload_file_to(n1, "configuration-files/ipfix_live_1.conf"))
print(cmd_upload_file_to(n2, "configuration-files/ipfix_live_1.conf"))

##### Execution

1. Move the configuration file to the pmacctd folder.
2. Start the pmacctd daemon using threads. 
3. Execute ICMP live traffic to the n1 so that it can be collected.
4. Show the results on the terminal.

In [ ]:
#Move configuration file to pmacctd directory, execute experiment and see results
n1.execute("mv ipfix_live_1.conf pmacct-1.7.8")
n2.execute("mv ipfix_live_1.conf pmacct-1.7.8")

In [ ]:
import threading 
def run_pmacctd(node, command):
    result = node.execute(command)
    print(result)
command_n1 = "cd pmacct-1.7.8 && sudo pmacctd -f ipfix_live_1.conf"
command_n2 = "ping -c 20 " + n1_ip

thread_h1 = threading.Thread(target=run_pmacctd, args=(n1, command_n1))
thread_h2 = threading.Thread(target=run_pmacctd, args=(n2, command_n2))

thread_h1.start()
thread_h2.start()

thread_h2.join()

### Print the result on a log file

For this second experiment we will use the following configuration file: 

daemonize: false \
debug: true \
pcap_interface: any \
plugins: nfprobe, print \
nfprobe_version: 10 \
print_output_file: ipfix_live_2.log

- print: This plugin is used for printing the collected data, in this case to a log file since we have specified an output file.
- pcap_interface: refers to the interface or interfaces where the tool will collect the data from.
- print_output_file: refers to the log output file.

In [ ]:
# You need to paste the resulting commands into the terminal to upload the files
print(cmd_upload_file_to(n1, "configuration-files/ipfix_live_2.conf"))
print(cmd_upload_file_to(n2, "configuration-files/ipfix_live_2.conf"))

##### Execution

1. Create the result result log file.
2. Move the configuration file and the result log file to the pmacctd folder.
3. Execute pmacctd.
4. Create live traffic betweeen the nodes.
5. Show the results.
6. You can execute the last cell of the section to download the .log result file into your computer.

In [ ]:
#Move configuration file to pmacctd directory, execute experiment and see results
n1.execute("touch ipfix_live_2.log")
n1.execute("mv ipfix_live_2.conf ipfix_live_2.log pmacct-1.7.8")
n2.execute("mv ipfix_live_2.conf pmacct-1.7.8")

In [ ]:
import threading 
def run_pmacctd(node, command):
    result = node.execute(command)
    print(result)
command_n1 = "cd pmacct-1.7.8 && sudo pmacctd -f ipfix_live_2.conf"
command_n2 = "ping -c 20 " + n1_ip

thread_h1 = threading.Thread(target=run_pmacctd, args=(n1, command_n1))
thread_h2 = threading.Thread(target=run_pmacctd, args=(n2, command_n2))

thread_h1.start()
thread_h2.start()

thread_h2.join()

In [ ]:
print(cmd_download_file_from(n1, "pmacct-1.7.8/ipfix_live_2.log"))

### Send the IPFIX flows to an .ipfix file 
For this experiment we will use the following configuration file: 

daemonize: false \
debug: true \
pcap_interface: any \
plugins: nfprobe \
nfprobe_receiver: 127.0.0.4:4738 \
nfprobe_version: 10 

- nfprobe_receiver: it configures pmacct to send the collected network flow data to a local collector on IP address 127.0.0.4 running on port 4739.

In [ ]:
# Please, paste the resulting commands to upload the configuration files to the nodes

print(cmd_upload_file_to(n1, "configuration-files/ipfix_live_3.conf"))
print(cmd_upload_file_to(n2, "configuration-files/ipfix_live_3.conf"))

##### Redirecting the flow of 127.0.0.4 port 4739 to the file. 
We are going to do that using netcat. So we first need to download it and then use the following command to redirect the ipfix flows created to it: nc -u -l 4739 > pmacct-1.7.8/ipfix_flows_live_3.ipfix.

This command means that we are using the NetCat utility using UDP which listens for incoming connections on port 4739 and which redirects the output to the file specified on the path, in this cas ipfix_flows_3.ipfix


In [98]:
stdout, stderr = n1.execute(
    "sudo apt-get install -y netcat",
    quiet=True
)
stdout, stderr = n2.execute(
    "sudo apt-get install -y netcat",
    quiet=True
)


##### Execution

1. Create the result result IPFIX file.
2. Move the configuration file and the result file to the pmacctd folder.
3. Redirect the network flow of 127.0.0.4 port 4739 to the file
4. Execute pmacctd.
5. Show the results.
6. You can execute the last cell of the section to download the .log result file into your computer.

In [ ]:
#Move configuration file to pmacctd directory, execute experiment and see results
n1.execute("touch ipfix_flows_live_3.ipfix")
n1.execute("mv ipfix_live_3.conf ipfix_flows_live_3.ipfix pmacct-1.7.8")
n2.execute("mv ipfix_live_3.conf pmacct-1.7.8")

In [ ]:
import threading

def run_iperf(node, command):
    result = node.execute(command)
    print(result)
command_1 = "nc -u -l 4738 > pmacct-1.7.8/ipfix_flows_live_3.ipfix"
command_2 = "cd pmacct-1.7.8 && sudo pmacctd -f ipfix_live_3.conf"
command_3 = "ping -c 20 -I " + n1_iface1.get_device_name() + n1_ip

thread_1 = threading.Thread(target=run_iperf, args=(n1, command_1))
thread_2 = threading.Thread(target=run_iperf, args=(n1, command_2))
thread_3 = threading.Thread(target=run_iperf, args=(n2, command_3))

thread_1.start()
thread_2.start()
thread_3.start()

thread_3.join()

In [ ]:
print(cmd_download_file_from(n1, "pmacct-1.7.8/ipfix_flows_live_3.ipfix"))

##### We need to download t-shark so that we can se the content of the flows in the result file. 
First, we need to download the file to inspect its content and the different flows. Since downloading TShark requires granting some permissions, we cannot automate this process. Please refer to the cell where the SSH command for node n1 is shown in the section. Execute that command in a terminal, and then run the following command: "sudo apt-get install -y tshark"


##### Summary
The following command will read and display the packet summary information from the IPFIX file, showing a list of packets captured in the file.


In [ ]:
n1.execute("tshark -r pmacct-1.7.8/ipfix_flows_live_3.ipfix")

##### In detail 
This command reads the IPFIX file and provides a detailed hex and ASCII representation of the first packet's data. This is useful for in-depth analysis of the packet's structure and content.

You can change the number of the packet and execute the command again to inspect different ones.


In [ ]:
# Look one packet of a flow in detai
n1.execute("tshark -r pmacct-1.7.8/ipfix_flows_live_3.ipfix -x -c 1")

### Send the IPFIX flows to a collector

First, we need to download the netflow Python library from  https://pypi.org/project/netflow/ and place on the work directory. 
Also it is necessary to download python3 on the node: sudo apt-get install python3 to be able to use the library. Then we will uploade it to the nodes. 

In [ ]:
# Upload the netflow library to the nodes.
print(cmd_upload_file_to(n1, "netflow-0.12.2.tar.gz"))
print(cmd_upload_file_to(n2, "netflow-0.12.2.tar.gz"))      

In [ ]:
# Unzip the netflow library file, you can install the zip library by executing the command sudo apt-get zip, you can uncomment the following line and execute it
# If you follow this notebook from the beggining you should have the software package already installed.
#n1.execute("sudo apt-get zip")
n1.execute("tar -xzf netflow-0.12.2.tar.gz")
n2.execute("tar -xzf netflow-0.12.2.tar.gz")

It is necessary to download python3 and pip on the node to be able to use the library.

In [ ]:
# Download pyython at the nodes and also netflow
stdout, stderr = n1.execute(
    "sudo apt-get install -y python3",
    quiet=True
)
stdout, stderr = n1.execute(
    "sudo apt-get install -y python3",
    quiet=True
)
stdout, stderr = n2.execute(
    "sudo apt-get install -y pip",
    quiet=True
)
stdout, stderr = n2.execute(
    "sudo apt-get install -y pip",
    quiet=True
)


We also need to install the netflow Python package so that the library works properly.

In [ ]:
stdout, stderr = n1.execute(
    "pip install netflow",
    quiet=True
)
stdout, stderr = n2.execute(
    "pip install netflow",
    quiet=True
)


##### Configuration file

For this experiment we will use the following configuration file: 

daemonize: false \
debug: true \
pcap_interface: any \
plugins: nfprobe \
nfprobe_receiver: 127.0.0.6:4739 \
nfprobe_version: 9 

- nfprobe_version: 9 refers to NetFlow, since we are using its Python version we need to specify it.

In [ ]:
# Please, paste the commands on the terminal to upload the files to the node
print(cmd_upload_file_to(n1, "configuration-files/ipfix_live_4.conf"))
print(cmd_upload_file_to(n2, "configuration-files/ipfix_live_4.conf"))

##### Execution

1. Move the configuration file to the pmacctd folder.
2. Execute the python program collector.py specifying the host and the port 
3. Redirect the network flow of 127.0.0.6 port 4739 to the file
4. Execute pmacctd daemon.
5. Create ICMP live traffic
6. Show the results.

In [ ]:
n1.execute("mv ipfix_live_4.conf pmacct-1.7.8")
n2.execute("mv ipfix_live_4.conf pmacct-1.7.8")

In [ ]:
import threading

def run_iperf(node, command):
    result = node.execute(command)
    print(result)
command_1 = "cd netflow-0.12.2/netflow && python3 collector.py --host 127.0.0.6 --port 4739"
command_2 = "cd pmacct-1.7.8 && sudo pmacctd -f ipfix_live_4.conf"
command_3 = "ping -c 40 -I " + n1_iface1.get_device_name() + n1_ip

thread_1 = threading.Thread(target=run_iperf, args=(n1, command_1))
thread_2 = threading.Thread(target=run_iperf, args=(n1, command_2))
thread_3 = threading.Thread(target=run_iperf, args=(n2, command_3))

thread_1.start()
thread_2.start()
thread_3.start()

thread_3.join()

##### Results

The results will be obtained on a .tar file, containing a JSON file placed on the netflow folder as you can see on the following picture:

<div style="text-align: center;">
    <img src="./images/4.png" width="700px"/>
</div>

You can unzip them using the command: gzip -d filename.gz

You can see the results using the command: cat filename

You can download the results using the command: print(cmd_download_file_from(n1, "filename"))


# Future work
This notebook focuses on the integration of pmacctd with FABRIC, but this is not its ultimate goal. The aim is to create an opportunity for future students and individuals interested in networking within the field of computer science to experiment and learn by leveraging the combined capabilities of these two tools. Therefore, I encourage users to continue contributing to this notebook by modifying experiments, tweaking the pmacctd configuration files, experimenting with different topologies, incorporating pmacctd into their own projects, or simply exploring the numerous functionalities that pmacct offers.